In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
import zipfile
import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.applications import VGG16
from keras.optimizers import Adam
from keras.preprocessing import image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
import numpy as np
import random

In [4]:
Seed=14
random.seed(Seed)
np.random.seed(Seed)
tf.random.set_seed(Seed)

In [5]:
batch_size=32
n_epochs=5
img_rows, img_cols= 244, 244
imput_shape= (img_rows, img_cols, 3)

In [6]:
import tarfile
import urllib.request
import os
import shutil

# URL of the tar file
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/ZjXM4RKxlBK9__ZjHBLl5A/aircraft-damage-dataset-v1.tar"

# Define the path to save the file
tar_filename = "aircraft_damage_dataset_v1.tar"
extracted_folder = "aircraft_damage_dataset_v1"  # Folder where contents will be extracted

# Download the tar file
urllib.request.urlretrieve(url, tar_filename)
print(f"Downloaded {tar_filename}. Extraction will begin now.")

# Check if the folder already exists
if os.path.exists(extracted_folder):
    print(f"The folder '{extracted_folder}' already exists. Removing the existing folder.")
    
    # Remove the existing folder to avoid overwriting or duplication
    shutil.rmtree(extracted_folder)
    print(f"Removed the existing folder: {extracted_folder}")

# Extract the contents of the tar file
with tarfile.open(tar_filename, "r") as tar_ref:
    tar_ref.extractall()  # This will extract to the current directory
    print(f"Extracted {tar_filename} successfully.")

Downloaded aircraft_damage_dataset_v1.tar. Extraction will begin now.
Extracted aircraft_damage_dataset_v1.tar successfully.


In [7]:
extract_path= "aircraft_damage_dataset_v1"
train_dir= os.path.join(extract_path, "train")
test_dir= os.path.join(extract_path, "test")
valid_dir= os.path.join(extract_path, "valid")

In [8]:
train_datagen= ImageDataGenerator(rescale=1./255)
test_datagen= ImageDataGenerator(rescale=1./255)
valid_datagen= ImageDataGenerator(rescale=1./255)

In [9]:
train_generator= train_datagen.flow_from_directory(
    train_dir,
    target_size= (img_rows, img_cols),
    batch_size= batch_size,
    seed= Seed,
    class_mode= "binary",
    shuffle= True
)

Found 300 images belonging to 2 classes.


In [10]:
test_generator= test_datagen.flow_from_directory(
    test_dir,
    target_size= (img_rows, img_cols),
    batch_size= batch_size,
    seed= Seed,
    class_mode= "binary",
    shuffle= False
)

Found 50 images belonging to 2 classes.


In [11]:
valid_generator= valid_datagen.flow_from_directory(
    valid_dir,
    batch_size= batch_size,
    target_size= (img_rows, img_cols),
    seed= Seed,
    class_mode= "binary",
    shuffle=False
)

Found 96 images belonging to 2 classes.


In [12]:
base_model= VGG16(weights= 'imagenet', include_top= False, input_shape=(img_rows, img_cols, 3))

In [14]:
output= base_model.layers[-1].output
output= keras.layers.Flatten()(output)
base_model= Model(base_model.input, output)

for layer in base_model.layers:
    layer.trainable = False